In [3]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np


# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

import imageio
from IPython.display import HTML, display


In [2]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [3]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[19:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/*")
    save_dir = "dataset"

    for path in video_paths:
        save_frame(path, save_dir, gap=30)

yoga_vid_collected\abhay_bhuj
abhay_bhuj
dataset\Bhujangasana\abhay_bhuj
yoga_vid_collected\abhay_padam
abhay_padam
dataset\Padamasana\abhay_padam
yoga_vid_collected\abhay_shav
abhay_shav
dataset\Shavasana\abhay_shav
yoga_vid_collected\abhay_tada
abhay_tada
dataset\Tadasana\abhay_tada
yoga_vid_collected\abhay_trik
abhay_trik
dataset\Trikonasana\abhay_trik
yoga_vid_collected\abhay_vriksh
abhay_vriksh
dataset\Vrikshasana\abhay_vriksh
yoga_vid_collected\ameya_bhuj
ameya_bhuj
dataset\Bhujangasana\ameya_bhuj
yoga_vid_collected\ameya_padam
ameya_padam
dataset\Padamasana\ameya_padam
yoga_vid_collected\ameya_shav
ameya_shav
dataset\Shavasana\ameya_shav
yoga_vid_collected\ameya_tada
ameya_tada
dataset\Tadasana\ameya_tada
yoga_vid_collected\ameya_trik
ameya_trik
dataset\Trikonasana\ameya_trik
yoga_vid_collected\ameya_vriksh
ameya_vriksh
dataset\Vrikshasana\ameya_vriksh
yoga_vid_collected\bhumi_bhuj
bhumi_bhuj
dataset\Bhujangasana\bhumi_bhuj
yoga_vid_collected\bhumi_padam
bhumi_padam
dataset\Pada

In [14]:
# helper function for visualization

#@title Helper function for visualization
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:

      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

def to_gif(images, fps):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, fps=fps)
  return embed.embed_file('./animation.gif')

def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

In [15]:
# Visualize the predictions with image.


image = tf.io.read_file('E:/BE/FinalYearProjectDetails/Yoga_Class-Analysis/dataset/Bhujangasana/abhay_bhuj/15.png')
image = tf.image.decode_png(image)
display_image = image
display_image = tf.cast(tf.image.resize_with_pad(
display_image, 1280, 1280), dtype=tf.int32)
output_overlay = draw_prediction_on_image(np.squeeze(display_image.numpy(), axis=0), keypoints_tup[0][0])
processed_image_array.append(output_overlay)
  # keypoint_array = keypoint_with_scores ->

  

plt.figure(figsize=(12,12))

for i in range(len(processed_image_array)):
  plt.subplot(3, 3, i + 1)
  plt.imshow(processed_image_array[i])
  plt.axis('off')
plt.show()

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [10]:
# Load the input image.
def image_to_movenet(path):
    
    if (path.find("Bhujangasana") != -1) :
          label="Bhujangasana"  
    elif (path.find("Padamasana") != -1):
          label="Padamasana"
    elif (path.find("Shavasana") != -1):
          label="Shavasana"
    elif (path.find("Trikonasana") != -1):
          label="Trikonasana"
    elif (path.find("Vrikshasana") != -1):
          label="Vrikshasana"
    elif (path.find("Tadasana") != -1):
          label="Tadasana"
        
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    keypoints_tup=(keypoints,label)
    keypts_labels.append(keypoints_tup)
    print(path)
    print(keypts_labels)
    
    

In [11]:
# import os
# import time

# path_lst=[]
# keypts_labels=[]
# class_list=os.listdir('dataset')
# for name in class_list:
#     for image in os.listdir(f"dataset/{name}"):
#         for item in os.listdir(f"dataset/{name}/{image}"):
# #             add_tup=(os.path.abspath(f"dataset/{name}/{image}/{item}"),name)
#             path_lst.append(os.path.abspath(f"dataset/{name}/{image}/{item}"))



# path_lst = sorted(path_lst,key=os.path.getmtime)
# for path in path_lst:
#     image_to_movenet(path)
path='E:/BE/FinalYearProjectDetails/Yoga_Class-Analysis/dataset/Bhujangasana/abhay_bhuj/15.png'
image_to_movenet(path)
    


E:/BE/FinalYearProjectDetails/Yoga_Class-Analysis/dataset/Bhujangasana/abhay_bhuj/15.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana'), (<tf.Tensor: shape